### using langsmith for debugging, monitoring 


In [1]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true' 
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')

### using document loaders to create a knowledge base base

In [2]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://www.moneycontrol.com/news/business/ratan-tata-death-news-live-updates-tata-group-chairman-last-rights-funeral-liveblog-12839261.html")

docs = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


### creating different chunks by splitting the docs

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
splits = text_splitter.split_documents(docs)


### creating embeddings for the docs and store it in a vector store

 https://www.rungalileo.io/blog/mastering-rag-how-to-select-an-embedding-model#:~:text=When%20selecting%20an%20embedding%20model,%2C%20and%20Voyager%2C%20among%20others.

In [4]:
from langchain_community.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings

embed = OllamaEmbeddings(
    model="llama3.2"
)
vector_store = Chroma.from_documents(documents=splits, embedding= embed)

In [5]:
print(len(vector_store))

32


### retriever

In [6]:
retriever = vector_store.as_retriever()

###  using ollama and meta3.2 3B

In [7]:
from langchain_ollama import ChatOllama
local_llm = "llama3.2"
llm = ChatOllama(model=local_llm, temperature=0)



In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {question}""")
parser = StrOutputParser()



In [9]:
from langchain_core.runnables import RunnablePassthrough

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)



### Question


In [10]:
rag_chain.invoke("What is Task Decomposition?")

'There is no mention of "Task Decomposition" in the provided context. The text appears to be about Ratan Tata\'s life, his childhood, and his passing away, but it does not discuss the concept of "Task Decomposition".'

In [11]:
rag_chain.invoke("Who is Ratan Tata")

"The article does not explicitly state who Ratan Tata is, but based on the context and the mentions of him in the articles, it can be inferred that Ratan Tata was a notable industrialist or businessperson. The articles mention his death, contributions to the Confederation of Indian Industry (CII), and quotes from other prominent individuals such as Viswanathan Anand, suggesting that he was a respected figure in India's industry and possibly a well-known public figure."

In [12]:
rag_chain.invoke("Which people are mentioned in the article")

'The following people are mentioned in the article:\n\n1. Ratan Tata\n2. Thackerays (referring to the Thackeray family, likely referring to Shiv Sena leaders Eknath Shinde and Uddhav Thackeray)\n3. Pawars (referring to the Pawar family, likely referring to Sharad Pawar or his relatives)'